# YOLOv5 + EfficientNet-B7 Research on Google Colab

This notebook demonstrates the integration of EfficientNet-B7 as a backbone for YOLOv5 and compares it with the baseline YOLOv5s model.

## 🎯 Objectives
1. Setup the research environment
2. Train baseline YOLOv5s model
3. Train EfficientNet-B7 + YOLOv5 model
4. Compare performance metrics
5. Visualize results

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Clone research repository
!git clone https://github.com/josh-jaiy/yolov5-efficientnet-research.git
%cd yolov5-efficientnet-research

In [ ]:
# Install dependencies
!pip install -r requirements.txt

# Clone YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

In [ ]:
# Setup custom EfficientNet integration
!cp ../models/efficientnet_adapter.py models/
!cp ../configs/yolov5s_effnet_multiscale.yaml models/
!cp ../modified_files/yolo.py models/

In [ ]:
# Test EfficientNet integration
!python ../scripts/test_backbone_final.py

In [ ]:
# Train baseline YOLOv5s model
!python train.py --data coco128.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --epochs 10 --name baseline_colab --batch-size 16

In [ ]:
# Train EfficientNet-B7 + YOLOv5 model (Memory Optimized)
print("🚀 Starting EfficientNet-B7 training with optimized settings...")
print("📊 Configuration:")
print("   - Epochs: 5 (reduced for memory efficiency)")
print("   - Batch size: 4 (reduced for memory constraints)")
print("   - Mixed precision: Enabled (--amp for memory savings)")
print("   - Expected training time: ~15-20 minutes")
print()

!python train.py --data coco128.yaml --cfg models/yolov5s_effnet_multiscale.yaml --epochs 5 --batch-size 4 --amp --name efficientnet_colab_optimized --device 0

print("\n✅ EfficientNet-B7 + YOLOv5 training completed!")
print("📈 Results saved to: runs/train/efficientnet_colab_optimized/")

In [ ]:
# Compare results
import pandas as pd
import matplotlib.pyplot as plt

# Load training results
baseline_results = pd.read_csv('runs/train/baseline_colab/results.csv')
efficientnet_results = pd.read_csv('runs/train/efficientnet_colab_optimized/results.csv')

print("📊 Training Results Summary:")
print(f"Baseline final mAP@0.5: {baseline_results['metrics/mAP_0.5'].iloc[-1]:.3f}")
print(f"EfficientNet final mAP@0.5: {efficientnet_results['metrics/mAP_0.5'].iloc[-1]:.3f}")
improvement = (efficientnet_results['metrics/mAP_0.5'].iloc[-1] / baseline_results['metrics/mAP_0.5'].iloc[-1] - 1) * 100
print(f"Performance improvement: {improvement:+.1f}%")
print()

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# mAP@0.5
axes[0,0].plot(baseline_results['epoch'], baseline_results['metrics/mAP_0.5'], label='Baseline YOLOv5s')
axes[0,0].plot(efficientnet_results['epoch'], efficientnet_results['metrics/mAP_0.5'], label='EfficientNet-B7')
axes[0,0].set_title('mAP@0.5')
axes[0,0].legend()

# Training loss
axes[0,1].plot(baseline_results['epoch'], baseline_results['train/box_loss'], label='Baseline YOLOv5s')
axes[0,1].plot(efficientnet_results['epoch'], efficientnet_results['train/box_loss'], label='EfficientNet-B7')
axes[0,1].set_title('Training Loss')
axes[0,1].legend()

plt.tight_layout()
plt.show()